In [ ]:
%cd ..

In [ ]:
import os

import cassandra.cluster
import cassandra.protocol
from pympler.asizeof import asizeof

import cassarrow
import cassarrow.impl

In [ ]:
QUERY = "SELECT * FROM cassarrow.time_series WHERE event_date = '2019-10-02'"

In [ ]:
cluster = cassandra.cluster.Cluster()
session = cluster.connect("cassarrow")

In [ ]:
def execute_default(session, query):
    results = session.execute(query)
    return list(results)

In [ ]:
def execute_cassarrow(session, query):
    with cassarrow.install_cassarrow(session) as cassarrow_session:
        results = cassarrow_session.execute(query)
    return cassarrow.result_set_to_table(results)

In [ ]:
%timeit execute_default(session, QUERY)

In [ ]:
%timeit execute_cassarrow(session, QUERY)

In [ ]:
def get_binary(name: str) -> bytes:
    full_name = os.path.join("tests", "select", name)
    with open(full_name, "rb") as fp:
        return fp.read()


data = get_binary("time_series/0011.bin")

In [ ]:
def parse_arrow(data):
    msg_arrow = cassarrow.impl.ArrowProtocolHandler.decode_message(
        5, {}, 3, 0, 8, data, None, []
    )
    return msg_arrow.parsed_rows


record_batch = parse_arrow(data)

In [ ]:
def parse_default(data):
    msg_arrow = cassandra.protocol._ProtocolHandler.decode_message(
        5, {}, 3, 0, 8, data, None, []
    )
    return msg_arrow.parsed_rows


tuples = parse_tuple(data)

In [ ]:
%timeit parse_arrow(data)

In [ ]:
%timeit parse_default(data)

In [ ]:
asizeof(tuples) / asizeof(record_batch)